In [1]:
from data import Data
from dimension_reduction import PCADimensionReduction
from simple_ml_models import *
from utils import *

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn import svm
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

2022-11-26 22:25:37.286778: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_filepath = "data/SC_integration/counts_ctc_simulated_123_5k.tsv"
true_results_filepath = "data/SC_integration/ids_ctc_simulated_123_5k.tsv"
train_indices_filepath = "data/SC_integration/train_indices.npy"
test_indices_filepath = "data/SC_integration/test_indices.npy"
SEED = 42
FOLD_NUMBER = 3

# You can change these values to work better for models
CUT_BY_MAX_THRESHOLD = 4
PCA_VARIABLES_AMOUNT = 60

# There are 4 data variants to check: regular, scaled, cut by max, pca reduced + cut by max
data_object = Data(data_filepath, true_results_filepath)
train_data, test_data, train_true_results, test_true_results =  data_object.load_train_test_split(train_indices_filepath, test_indices_filepath)
scaled_train_data, scaled_test_data = data_object.get_scaled_train_test_data()

In [3]:
cut_by_max_train_data, cut_by_max_test_data = data_object.get_cut_by_max_train_test_data(CUT_BY_MAX_THRESHOLD)

pca_object = PCADimensionReduction(cut_by_max_train_data, scaled_train_data, train_true_results, SEED)
pca_variables = pca_object.get_most_important_variables_from_pc1(PCA_VARIABLES_AMOUNT)
pca_reduced_train_data = cut_by_max_train_data[pca_variables.index]
pca_reduced_test_data = cut_by_max_test_data[pca_variables.index]

### Defined classifiers

In [4]:
# Logistic Regression
log_clf = LogisticRegression(random_state=SEED,
                             class_weight='balanced',
                             penalty='l1',
                             C=0.0005,
                             solver='liblinear')

# XGBoost
xgb_clf = XGBClassifier(random_state=SEED,
                        booster='gbtree',
                        eta=0.2,
                        min_child_weight=1,
                        max_depth=5,
                        gamma=0.5)

# K-Nearest Neighbors
knn_clf = KNeighborsClassifier(n_neighbors=5,
                               algorithm='kd_tree',
                               weights='distance')

# Isolation Forest
if_clf = IsolationForest(max_features=1,
                         n_estimators=1000,
                         bootstrap=True,
                         max_samples=1000,
                         n_jobs=12,
                         random_state=SEED)

# SVM
svm_clf = svm.SVC(kernel='linear',
                  class_weight='balanced',
                  probability=True,
                  random_state=SEED)

# Light GBM
lgbm_clf = LGBMClassifier(boosting_type='gbdt',
                          min_child_weight=0.001,
                          max_depth=10,
                          random_state=SEED,)

# Random Forest
rf_clf = RandomForestClassifier(random_state=SEED,
                                n_estimators=2000,
                                criterion="log_loss")

# Balanced Random Forest
brf_clf = BalancedRandomForestClassifier(random_state=SEED,
                                         n_estimators=2000)

In [5]:
names = ["Logistic Regression", "XGBoost", "K-Nearest Neighbors", "Isolated Forest",
         "SVM", "Light GBM", "Random Forest", "Balanced Random Forest"]
    
classifiers = [log_clf, xgb_clf, knn_clf, if_clf, svm_clf, lgbm_clf, rf_clf, brf_clf]

### Regular data

In [6]:
run_all_models(classifiers, names, train_data, train_true_results, test_data, test_true_results)

Features number: 2000
Best balanced accuracy: Isolated Forest, Random Forest, Balanced Random Forest
Best ROC AUC: XGBoost, Isolated Forest, Light GBM, Random Forest, Balanced Random Forest
Best precision: Isolated Forest, Random Forest, Balanced Random Forest
Best recall: Isolated Forest, Random Forest, Balanced Random Forest
Best F1 score: Isolated Forest, Random Forest, Balanced Random Forest


,Balanced Accuracy,ROC AUC,Precision,Recall,F1 score
Logistic Regression,0.500000,0.500000,0.977368,0.988619,0.982961
XGBoost,0.922222,1.000000,0.998234,0.998230,0.998111
K-Nearest Neighbors,0.500000,0.500000,0.977368,0.988619,0.982961
Isolated Forest,1.000000,1.000000,1.000000,1.000000,1.000000
SVM,0.500000,0.806447,0.977368,0.988619,0.982961
Light GBM,0.944444,1.000000,0.998738,0.998735,0.998676
Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000
Balanced Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000


### Scaled data

In [7]:
run_all_models(classifiers, names, scaled_train_data, train_true_results, scaled_test_data, test_true_results)

Features number: 2000
Best balanced accuracy: Isolated Forest, Random Forest, Balanced Random Forest
Best ROC AUC: XGBoost, Isolated Forest, Light GBM, Random Forest, Balanced Random Forest
Best precision: Isolated Forest, Random Forest, Balanced Random Forest
Best recall: Isolated Forest, Random Forest, Balanced Random Forest
Best F1 score: Isolated Forest, Random Forest, Balanced Random Forest


,Balanced Accuracy,ROC AUC,Precision,Recall,F1 score
Logistic Regression,0.966667,0.933419,0.999242,0.999241,0.999228
XGBoost,0.922222,1.000000,0.998234,0.998230,0.998111
K-Nearest Neighbors,0.500000,0.499488,0.977368,0.988619,0.982961
Isolated Forest,1.000000,1.000000,1.000000,1.000000,1.000000
SVM,0.610599,0.775799,0.987264,0.990137,0.987361
Light GBM,0.966667,1.000000,0.999242,0.999241,0.999219
Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000
Balanced Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000


### Cut by max data

In [8]:
run_all_models(classifiers, names, cut_by_max_train_data, train_true_results, cut_by_max_test_data, test_true_results)

Features number: 66
Best balanced accuracy: Balanced Random Forest
Best ROC AUC: XGBoost, Light GBM, Random Forest
Best precision: Light GBM
Best recall: Light GBM
Best F1 score: Light GBM


,Balanced Accuracy,ROC AUC,Precision,Recall,F1 score
Logistic Regression,0.500000,0.500000,0.977368,0.988619,0.982961
XGBoost,0.977778,1.000000,0.999495,0.999494,0.999486
K-Nearest Neighbors,0.522222,0.742637,0.985450,0.989125,0.984160
Isolated Forest,0.980941,0.999966,0.991426,0.962317,0.973832
SVM,0.695344,0.709798,0.982844,0.918816,0.947847
Light GBM,0.988889,1.000000,0.999747,0.999747,0.999743
Random Forest,0.944444,1.000000,0.998737,0.998735,0.998695
Balanced Random Forest,0.998209,0.999983,0.997362,0.996459,0.996717


### Pca reduced + cut by max data

In [9]:
run_all_models(classifiers, names, pca_reduced_train_data, train_true_results, pca_reduced_test_data, test_true_results)

Features number: 60
Best balanced accuracy: XGBoost, Light GBM
Best ROC AUC: XGBoost, Light GBM, Random Forest
Best precision: XGBoost, Light GBM
Best recall: XGBoost, Light GBM
Best F1 score: XGBoost, Light GBM


,Balanced Accuracy,ROC AUC,Precision,Recall,F1 score
Logistic Regression,0.500000,0.500000,0.977368,0.988619,0.982961
XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000
K-Nearest Neighbors,0.555556,0.809363,0.989987,0.989884,0.985800
Isolated Forest,0.955615,0.999881,0.989953,0.912241,0.945021
SVM,0.669745,0.472516,0.982242,0.846485,0.906305
Light GBM,1.000000,1.000000,1.000000,1.000000,1.000000
Random Forest,0.955556,1.000000,0.998990,0.998988,0.998953
Balanced Random Forest,0.997058,0.999983,0.996206,0.994183,0.994790


### P-values regular data

In [10]:
statistics = calculate_statistics(train_data, train_true_results.values.ravel())
statistics = statistics[statistics["p_values"] < 0.05]
p_values_regular_train_data = train_data[statistics.index]
p_values_regular_test_data = test_data[statistics.index]

run_all_models(classifiers, names, p_values_regular_train_data, train_true_results, p_values_regular_test_data, test_true_results)

Features number: 442
Best balanced accuracy: Isolated Forest, Random Forest, Balanced Random Forest
Best ROC AUC: XGBoost, Isolated Forest, Light GBM, Random Forest, Balanced Random Forest
Best precision: Isolated Forest, Random Forest, Balanced Random Forest
Best recall: Isolated Forest, Random Forest, Balanced Random Forest
Best F1 score: Isolated Forest, Random Forest, Balanced Random Forest


,Balanced Accuracy,ROC AUC,Precision,Recall,F1 score
Logistic Regression,0.500000,0.500000,0.977368,0.988619,0.982961
XGBoost,0.966667,1.000000,0.999242,0.999241,0.999228
K-Nearest Neighbors,0.500000,0.562292,0.977368,0.988619,0.982961
Isolated Forest,1.000000,1.000000,1.000000,1.000000,1.000000
SVM,0.533333,0.738808,0.985698,0.989378,0.984703
Light GBM,0.966667,1.000000,0.999242,0.999241,0.999228
Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000
Balanced Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000


### P-values scaled data

In [11]:
p_values_scaled_train_data = scaled_train_data[statistics.index]
p_values_scaled_test_data = scaled_test_data[statistics.index]

run_all_models(classifiers, names, p_values_scaled_train_data, train_true_results, p_values_scaled_test_data, test_true_results)

Features number: 442
Best balanced accuracy: Isolated Forest, Random Forest, Balanced Random Forest
Best ROC AUC: XGBoost, Isolated Forest, Light GBM, Random Forest, Balanced Random Forest
Best precision: Isolated Forest, Random Forest, Balanced Random Forest
Best recall: Isolated Forest, Random Forest, Balanced Random Forest
Best F1 score: Isolated Forest, Random Forest, Balanced Random Forest


,Balanced Accuracy,ROC AUC,Precision,Recall,F1 score
Logistic Regression,0.966667,0.933419,0.999242,0.999241,0.999228
XGBoost,0.966667,1.000000,0.999242,0.999241,0.999228
K-Nearest Neighbors,0.577650,0.721821,0.989531,0.990137,0.986694
Isolated Forest,1.000000,1.000000,1.000000,1.000000,1.000000
SVM,0.788889,0.714607,0.995218,0.995195,0.994535
Light GBM,0.966667,1.000000,0.999242,0.999241,0.999228
Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000
Balanced Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000


### P-values cut by max

In [12]:
statistics = calculate_statistics(cut_by_max_train_data, train_true_results.values.ravel())
statistics = statistics[statistics["p_values"] < 0.05]
p_values_cut_by_max_train_data = train_data[statistics.index]
p_values_cut_by_max_test_data = test_data[statistics.index]

run_all_models(classifiers, names, p_values_cut_by_max_train_data, train_true_results, p_values_cut_by_max_test_data, test_true_results)

Features number: 7
Best balanced accuracy: Random Forest
Best ROC AUC: XGBoost, Light GBM, Random Forest
Best precision: Random Forest
Best recall: Random Forest
Best F1 score: Random Forest


,Balanced Accuracy,ROC AUC,Precision,Recall,F1 score
Logistic Regression,0.500000,0.500000,0.977368,0.988619,0.982961
XGBoost,0.977778,1.000000,0.999495,0.999494,0.999476
K-Nearest Neighbors,0.710855,0.975544,0.992571,0.992919,0.991525
Isolated Forest,0.979534,0.997374,0.991143,0.959535,0.972082
SVM,0.604630,0.682101,0.980383,0.869752,0.919997
Light GBM,0.999872,1.000000,0.999763,0.999747,0.999751
Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000
Balanced Random Forest,0.991558,0.999744,0.993233,0.983308,0.986768


### P-values pca reduced

In [13]:
statistics = calculate_statistics(pca_reduced_train_data, train_true_results.values.ravel())
statistics = statistics[statistics["p_values"] < 0.05]
p_values_pca_reduced_train_data = train_data[statistics.index]
p_values_pca_reduced_test_data = test_data[statistics.index]

run_all_models(classifiers, names, p_values_pca_reduced_train_data, train_true_results, p_values_pca_reduced_test_data, test_true_results)

Features number: 6
Best balanced accuracy: XGBoost
Best ROC AUC: XGBoost, Light GBM, Random Forest
Best precision: XGBoost
Best recall: XGBoost
Best F1 score: XGBoost


,Balanced Accuracy,ROC AUC,Precision,Recall,F1 score
Logistic Regression,0.500000,0.500000,0.977368,0.988619,0.982961
XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000
K-Nearest Neighbors,0.710472,0.974691,0.991585,0.992160,0.990855
Isolated Forest,0.951778,0.996180,0.989854,0.904654,0.940696
SVM,0.597723,0.672312,0.980206,0.856095,0.912116
Light GBM,0.999872,1.000000,0.999763,0.999747,0.999751
Random Forest,0.988889,1.000000,0.999747,0.999747,0.999743
Balanced Random Forest,0.992453,0.999471,0.993550,0.985078,0.987980


### Logistic regression coefficients

In [21]:
logistic_regression_features = pd.read_csv("features/logistic_regression.csv", index_col=0)
logistic_regression_features = logistic_regression_features[(logistic_regression_features != 0).all(1)]
logistic_regression_features = logistic_regression_features.sort_values(by="feature_importance_vals", ascending=False)

lg_train_data = train_data[logistic_regression_features.index[:25]]
lg_test_data = test_data[logistic_regression_features.index[:25]]
run_all_models(classifiers, names, lg_train_data, train_true_results, lg_test_data, test_true_results)

Features number: 25
Best balanced accuracy: Light GBM, Balanced Random Forest
Best ROC AUC: XGBoost, Isolated Forest, Light GBM, Random Forest, Balanced Random Forest
Best precision: Light GBM, Balanced Random Forest
Best recall: Light GBM, Balanced Random Forest
Best F1 score: Light GBM, Balanced Random Forest


,Balanced Accuracy,ROC AUC,Precision,Recall,F1 score
Logistic Regression,0.500000,0.500000,0.977368,0.988619,0.982961
XGBoost,0.966667,1.000000,0.999242,0.999241,0.999219
K-Nearest Neighbors,0.500000,0.500000,0.977368,0.988619,0.982961
Isolated Forest,0.999616,1.000000,0.999289,0.999241,0.999253
SVM,0.698081,0.628106,0.988028,0.989378,0.988551
Light GBM,1.000000,1.000000,1.000000,1.000000,1.000000
Random Forest,0.988889,1.000000,0.999747,0.999747,0.999743
Balanced Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000


### PCA

In [29]:
pca_features = pd.read_csv("features/pca.csv", index_col=0)
pca_features = pca_features[(pca_features != 0).all(1)]
pca_features.sort_values(by="feature_importance_vals", ascending=False)

pca_train_data = train_data[pca_features.index[:100]]
pca_test_data = test_data[pca_features.index[:100]]
run_all_models(classifiers, names, pca_train_data, train_true_results, pca_test_data, test_true_results)

Features number: 100
Best balanced accuracy: Balanced Random Forest
Best ROC AUC: XGBoost, Isolated Forest, Light GBM, Random Forest, Balanced Random Forest
Best precision: Light GBM
Best recall: Light GBM
Best F1 score: Light GBM


,Balanced Accuracy,ROC AUC,Precision,Recall,F1 score
Logistic Regression,0.500000,0.500000,0.977368,0.988619,0.982961
XGBoost,0.900000,1.000000,0.997730,0.997724,0.997563
K-Nearest Neighbors,0.644444,0.806788,0.991973,0.991907,0.989681
Isolated Forest,0.919417,1.000000,0.989384,0.840668,0.903327
SVM,0.570964,0.507018,0.979524,0.933485,0.955231
Light GBM,0.977522,1.000000,0.999049,0.998988,0.998986
Random Forest,0.933333,1.000000,0.998485,0.998483,0.998419
Balanced Random Forest,0.998465,1.000000,0.997621,0.996965,0.997147


### XGBoost

In [ ]:
pca_reduced_xgboost_features = pd.read_csv("features/pca_reduced_xgboost.csv", index_col=0)
pca_reduced_xgboost_features = pca_reduced_xgboost_features[(pca_reduced_xgboost_features != 0).all(1)]
pca_reduced_xgboost_features.sort_values(by="feature_importance_vals", ascending=False)

xgb_train_data = train_data[pca_reduced_xgboost_features.index]
xgb_test_data = test_data[pca_reduced_xgboost_features.index]
run_all_models(classifiers, names, xgb_train_data, train_true_results, xgb_test_data, test_true_results)